# Trainingsdaten

Um überwachtes Lernen durchführen zu können, benötigst du eine sog. Trainingsmenge. Anhand dieser Menge von bereits klassifizierten Daten kann das Modell lernen, wie es unbekannte Daten klassifizieren kann.

Ob das funktioniert, hängt stark von der Güte der Trainingsmenge ab. Die Trainingsmenge sollte sauber und *nachvollziehbar* klassifiziert sein, dabei aber auch besonders *ausgewogen*, d.h. die unterschiedlichen Ergebnisse sollten in ungefähr der gleichen Anzahl vorkommen.

Damit das gelingt, kannst du auf bereits *explizit vorklassifizierte* Daten zurückgreifen oder auch mit *implizit klassifiziserten* Daten arbeiten. Schließlich kannst du Daten auch *manuell klassifizieren*.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

## Kategorien identifizieren

Um dir den Aufwand für die (fehleranfällige) manuelle Klassifikation zu ersparen, kannst du zunächst in den Daten nach geeigneten Feldern ausschau halten, die als Kategorien dienen könnten. Betrachte dazu zunächst alle Spalten des `DataFrame`:

In [ ]:
df.info()

Felder wie `title`, `header` und die gesamten linguistisch analysierten Daten enthalten ausschließlich Wörter und sind daher als Kategorien ungeeignet.

Anders sieht es mit den Feldern `author`, `keywords` und `commentCount` aus. Diese enthalten kategorische (teilweise mehrwertige) oder numerische Werte:

In [ ]:
df[["title", "author", "keywords", "commentCount"]]

Alle diese Felder kommen damit prinzipiell als Kategorien in Frage. Betrachte sie nun nacheinander:

## Kandidat: `author`

Du benögtigst mindestens ein paar Hundert Datensätze, um den Klassifikator zu trainieren. Wenn du für Artikel die Autoren vorhersagen möchtest, müsstest du daher Autoren finden, die mindestens 100 Artikel geschrieben haben. Betrachte dazu die Top-20-Autoren:

In [ ]:
df.groupby("author").count().sort_values("title", ascending=False).head(20)[["title"]]

Dies Autoren scheinen wirklich gut als Kategorien geeignet. Es handelt sich dabei um ein *Multi-Class Single-Label*-Problem, denn es gibt mehrere mögliche Autoren, aber jeder Artikel wurde nur von einem einzigen Autor geschrieben.

## Kandidat: `keywords`

Auch die Keywords könnten eine gute Wahl für eine bereits vorklassifizierte Kategorie sein. Allerdings sind pro Meldung mehrere Keywords vergeben. Wenn du die zählen möchtest, machst du das am besten mit einem `Counter`:

In [ ]:
from collections import Counter
keywords = Counter([keyword for keywords in df["keywords"] for keyword in str(keywords).split(", ")])

Betrachte nun die Top-20-Keywords:

In [ ]:
keywords.most_common(20)

Auch diese wurden häufig genug verwendet, so dass du dafür Beispiele finden kannst. In diesem Fall hast du ein *Multi Label*-Problem, weil für jeden Artikel mehrere Keywords vergeben sein könnten.

## Kandidat: `commentCount`

Die genaue Anzahl von Kommentaren zu prädizieren, wäre eine sog. *Regression*. Das wird sicher nicht klappen, weil das von zu vielen Einflussfaktoren abhängt. Aber du kannst versuchen, vorherzusagen ob ein Artikel viele oder wenige Kommentare auf sich zieht.

Betrachte dazu zunächst die Verteilung der Kommentare:

In [ ]:
df["commentCount"].dropna().map(int).plot.hist(bins=40)

Leider ist die Verteilung sehr verzerrt, weil es wenige Artikel mit sehr vielen Kommentaren gibt. Um das zu lösen, kannst du mit einem sog. *Cutoff* arbeiten und  die Anzahl der Kommentare in besonders erfolgreichen Artikeln auf 500 begrenzen:

In [ ]:
df["normalizedCommentCount"] = df["commentCount"].fillna(0).map(int)
df.loc[df["normalizedCommentCount"]>500, "normalizedCommentCount"] = 500

Die Verteilung sieht damit deutlich *ausgewogener* aus.

In [ ]:
df["normalizedCommentCount"].plot.hist(bins=40)

Jetzt könntest du *willkürlich* definieren, dass Artikel mit weniger als 10 Kommentaren in eine Klasse fallen sollen und solch mit mehr als 50 in eine andere. Achtung, in solchen Fällen solltest du einen *Sicherheitsabstand* einhalten, sonst überforderst du dein Modell auf jeden Fall:

In [ ]:
print(len(df[df["normalizedCommentCount"]<10]))
print(len(df[df["normalizedCommentCount"]>50]))

Wenn du klassifizieren möchtest, ob ein Artikel erfolgreich ist oder nicht, ist das ein sog. *Single Class*-Problem - es gibt als Ergebnis nur ja oder nein.

## Möglichst existierende Klassen nutzen

Daten manuell zu klassifizieren (*zu labeln*) ist sehr mühsam und extrem fehleranfällig. Crowdworker können dabei helfen.

Viel geschickter ist es allerdings, vorhandene Klassen zu nutzen. Diese müssen nicht immer explizit in den Daten enthalten sein, sondern evtl. kannst du auch implizite Klassen nutzen, wie z.B. die Anzahl der Kommentare.